In [ ]:
from math import floor

def max_pool(height, kernel_size, stride=None, padding=0, dilation=1):
    if stride is None:
        stride = kernel_size
    return floor((height + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)

def conv(height, kernel_size, stride=1, padding=0, dilation=1):
    return floor((height + 2 * padding - dilation * (kernel_size - 1) - 1) / stride + 1)

In [ ]:
sizes = [conv(400, 3)]
for i in range(0, 8, 2):
    sizes.append(max_pool(sizes[i], 2, stride=2))
    sizes.append(conv(sizes[i+1], 3))
sizes.append(max_pool(sizes[8], 2, stride=2))
sizes

In [ ]:
[s ** 2 for s in sizes]

In [1]:
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [2]:
data_dir = os.path.join("fich_database", "inputs")
dataset = datasets.ImageFolder(data_dir, transforms.ToTensor())

In [3]:
sizes = list()
for image, label in dataset:
    s = transforms.functional.get_image_size(image)
    sizes.append(s)

In [4]:
import statistics as stat

In [5]:
def stats(sizes, f):
    f0 = f([s[0] for s in sizes])
    f1 = f([s[1] for s in sizes])
    return f0, f1

In [6]:
stats(sizes, stat.stdev)

(1563.5263642279704, 1395.9083509326292)

In [7]:
stats(sizes, stat.mean)

(1521.471254095919, 1349.5555555555557)

In [8]:
stats(sizes, min)

(270, 270)

In [9]:
stats(sizes, max)

(6684, 6684)

In [10]:
import pickle

In [11]:
with open('image_sizes.pkl', 'wb') as file:
    pickle.dump(sizes, file)

In [12]:
with open('image_sizes.pkl', 'rb') as file:
    loaded_object = pickle.load(file)

In [15]:
stats(loaded_object, stat.stdev)

(1563.5263642279704, 1395.9083509326292)

In [ ]:
def resize(image):
    # noinspection PyUnresolvedReferences
    size = torch.tensor(transforms.functional.get_image_size(image))
    if all(size >= 400) and any(size <= 800):
        return image
    new_size = (size * torch.ceil(600 / torch.min(size))).int().tolist()
    t = transforms.Resize(new_size, antialias=True)
    return t(image)